In [1]:
from mapmos.live_m1p.lidar_new import RobosenseClientWrapper

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
a = RobosenseClientWrapper()

In [3]:
b = a.start()
if not b:
    print(b)
    print("✅ [Lidar] process started, waiting for point clouds.")
# a.get_point_cloud_numpy()

✅ Lidar client initialized successfully.


Got error: ERRCODE_PKTBUFOVERFLOW
Got error: ERRCODE_PKTBUFOVERFLOW
Got error: ERRCODE_PKTBUFOVERFLOW
Got error: ERRCODE_PKTBUFOVERFLOW
Got error: ERRCODE_PKTBUFOVERFLOW
Got error: ERRCODE_PKTBUFOVERFLOW
Got error: ERRCODE_PKTBUFOVERFLOW
Got error: ERRCODE_PKTBUFOVERFLOW
Got error: ERRCODE_PKTBUFOVERFLOW


In [1]:
from mapmos.live_m1p.lidar_new import LidarPipeline

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
a = LidarPipeline()


In [ ]:
a.save_point_clouds()

In [3]:
a.get_array()
# a.get_array()[0]

array([[        nan,         nan,         nan,  0.        ],
       [        nan,         nan,         nan,  0.        ],
       [        nan,         nan,         nan,  0.        ],
       ...,
       [        nan,         nan,         nan,  0.        ],
       [ 0.35093662,  0.44497937,  1.8449218 , 97.        ],
       [ 1.4077511 ,  0.52768093,  1.9911842 , 10.        ]],
      shape=(78750, 4), dtype=float32)

In [6]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
# Lade die PLY-Datei
pcd = o3d.io.read_point_cloud("/home/tguenther/repos/MapMOS/results/2025-09-29_13-35-55/ply/000093.ply")

# Prüfe die Daten
print(pcd)
print(f"Anzahl der Punkte: {len(pcd.points)}")

# Zeige die Punktwolke
o3d.visualization.draw_geometries([pcd])

PointCloud with 13398 points.
Anzahl der Punkte: 13398


In [15]:
pts = np.asarray(pcd.points)
pts.shape

(13398, 3)

In [32]:
punkte = np.asarray(pcd.points)   # Shape: (N, 3)
farben = np.asarray(pcd.colors)   # Shape: (N, 3)

In [2]:
import numpy as np
np.zeros((-1,3))

ValueError: negative dimensions are not allowed

In [1]:
import os
import hashlib
from PIL import Image
from concurrent.futures import ProcessPoolExecutor
from collections import defaultdict


In [2]:
# path = "/home/tguenther/Data/ato_recordings/2025-09-09_Records_CITEC/test_Thomas/Dataset_2025_09_09-13_56_12_Genie_M1P/M1P/velodyne_npy/"
path = "/home/tguenther/Data/ato_recordings/2025-09-09_Records_CITEC/test_Thomas/Dataset_2025_09_09-13_58_54_Genie_M1P/M1P/velodyne_npy/"
# path = "/home/tguenther/Data/ato_recordings/2025-09-09_Records_CITEC/test_Thomas/Dataset_2025_09_09-14_01_48_Genie_M1P/M1P/velodyne_npy/"


# datei1 = "000209_00112.96492.npy"
# datei2 = "000211_00114.03717.npy"#"000210_00113.50064.npy"

# # Dateien laden
# arr1 = np.load(path + datei1)
# arr2 = np.load(path + datei2)

# # Vergleichen
# gleich = np.array_equal(arr1, arr2)

# print(gleich)


In [3]:
ordner = path
# Alle npy-Dateien sammeln
dateien = [f for f in os.listdir(ordner) if f.endswith(".npy")]

In [5]:
dateien[3]

'000021_00009.87815.npy'

In [15]:
ordner = path
# Alle npy-Dateien sammeln
dateien = [f for f in os.listdir(ordner) if f.endswith(".npy")]

# Paare von Dateien vergleichen
for d1, d2 in combinations(dateien, 2):
    arr1 = np.load(os.path.join(ordner, d1))
    arr2 = np.load(os.path.join(ordner, d2))

    if np.array_equal(arr1, arr2):
        print(f"{d1} und {d2} sind identisch.")

In [11]:
basisordner = "/home/tguenther/Data/ato_recordings/2025-09-09_Records_CITEC/Dataset_2025_09_09-14_01_48_Genie_M1P/data/Genie/"

def finde_rgb_pngs(root):
    for r, _, files in os.walk(root):
        for f in files:
            if f.lower() == "rgb.png":
                yield os.path.join(r, f)

def hash_image(path):
    # Nur die tatsächlichen Pixel + Format/Größe hashen (Metadaten egal)
    with Image.open(path) as im:
        im.load()  # vollständiges Laden (verhindert lazy loading)
        # Größe + Modus mit in den Hash, damit unterschiedliche Modi/Sizes ungleich werden
        header = f"{im.mode}|{im.size[0]}x{im.size[1]}".encode("utf-8")
        pixel_bytes = im.tobytes()  # schneller als np.array(...).tobytes()
    h = hashlib.sha256()
    h.update(header)
    h.update(pixel_bytes)
    return (path, h.hexdigest())

def gruppiere_duplikate(dateipfade, workers=None, chunksize=32):
    dupe_map = defaultdict(list)
    with ProcessPoolExecutor(max_workers=workers) as ex:
        for path, digest in ex.map(hash_image, dateipfade, chunksize=chunksize):
            dupe_map[digest].append(path)
    # nur Gruppen mit >=2 Dateien
    return [pfade for pfade in dupe_map.values() if len(pfade) > 1]

if __name__ == "__main__":
    dateien = list(finde_rgb_pngs(basisordner))
    print(f"Gefundene PNGs: {len(dateien)}")
    dupe_gruppen = gruppiere_duplikate(dateien)  # nutzt alle Kerne
    if not dupe_gruppen:
        print("Keine Duplikate gefunden.")
    else:
        print("Duplikat-Gruppen:")
        for i, grp in enumerate(dupe_gruppen, 1):
            print(f"Gruppe {i}:")
            for p in grp:
                print("  ", p)

Gefundene PNGs: 233
Duplikat-Gruppen:
Gruppe 1:
   /home/tguenther/Data/ato_recordings/2025-09-09_Records_CITEC/Dataset_2025_09_09-14_01_48_Genie_M1P/data/Genie/000135_00074.74024/rgb.png
   /home/tguenther/Data/ato_recordings/2025-09-09_Records_CITEC/Dataset_2025_09_09-14_01_48_Genie_M1P/data/Genie/000136_00075.27399/rgb.png
   /home/tguenther/Data/ato_recordings/2025-09-09_Records_CITEC/Dataset_2025_09_09-14_01_48_Genie_M1P/data/Genie/000134_00074.20694/rgb.png
   /home/tguenther/Data/ato_recordings/2025-09-09_Records_CITEC/Dataset_2025_09_09-14_01_48_Genie_M1P/data/Genie/000133_00073.67330/rgb.png
Gruppe 2:
   /home/tguenther/Data/ato_recordings/2025-09-09_Records_CITEC/Dataset_2025_09_09-14_01_48_Genie_M1P/data/Genie/000192_00105.45032/rgb.png
   /home/tguenther/Data/ato_recordings/2025-09-09_Records_CITEC/Dataset_2025_09_09-14_01_48_Genie_M1P/data/Genie/000191_00104.91554/rgb.png
   /home/tguenther/Data/ato_recordings/2025-09-09_Records_CITEC/Dataset_2025_09_09-14_01_48_Genie_M1P/